# Load CSVs to Postgres

Workflow: 6 

Goal: Load all source files, TMDB, Reelgood, IMDB to database.

Result: The 3 database _Load tables are populated.

In [1]:
import os, sys
import time
import pandas as PD

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
os.chdir(os.getenv('PWD'))
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings.py")
import django
django.setup()

'app_proj.settings'

In [4]:
import app_proj.settings as ST
import movies.models.models as MD

## Read & Insert CSVs

In [5]:
MOVIEDB_FILE = os.path.join(ST.BASE_DIR, 'movies/data/moviedb_subsample.csv')
REELGOOD_FILE = os.path.join(ST.BASE_DIR, 'movies/data/reelgood_subsample.csv')
IMDB_FILE = os.path.join(ST.BASE_DIR, 'movies/data/imdb_subsample.csv')

In [6]:
MD.Editor.DeleteTable('All')

In [7]:
# the movie DB

moviedb_ls = MD.Editor.CSVtoDict(MOVIEDB_FILE)
len(moviedb_ls)

319

In [8]:
#moviedb_ls[12]

In [9]:
MD.Editor.InsertDictToTable(moviedb_ls, 'MovieDB_Load')
MD.MovieDB_Load.objects.all().count()

319

In [10]:
# reelgood

reelgood_ls = MD.Editor.CSVtoDict(REELGOOD_FILE)
len(reelgood_ls)

325

In [11]:
#reelgood_ls[90]

In [12]:
MD.Editor.InsertDictToTable(reelgood_ls, 'Reelgood_Load')
MD.Reelgood_Load.objects.all().count()

325

In [13]:
# imdb

imdb_ls = MD.Editor.CSVtoDict(IMDB_FILE)
len(imdb_ls)

317

In [14]:
#imdb_ls[0]

In [15]:
MD.Editor.InsertDictToTable(imdb_ls, 'IMDB_Load')
MD.IMDB_Load.objects.all().count()

317

## Create Master Movie Table

In [16]:
master_ls = MD.Editor.RunMasterMovies()
MD.Editor.InsertDictToTable(master_ls, 'MasterMovie')
MD.MasterMovie.objects.all().count()

319

In [17]:
master_df = PD.DataFrame(master_ls)
master_df.head()
master_df.info()

,Movie_ID,Title,OriginalTitle,Year,Rating,Companies,Country,Language,RunTime,Crew,Cast,Poster,Genres,Synopsis,Budget,Gross,ScoreImdb,VotesImdb,Indeces
0,217316,1,1,2013,PG-13,"Exclusive Media, Flat-Out Films",USA,English,112,"Mark Monroe, Paul Crowder","Niki Lauda, Lewis Hamilton",/4uIPXX8DjTsCzUAdtMKHTpojYLq.jpg,"Documentary, Sport, History",Set in the golden era of Grand Prix Racing '1'...,NaN,NaN,8.0,3535.0,"{""reelgood"": ""1-2013"", ""amazon"": ""B0751Q3J8P"",..."
1,386666,13 Times Evil,13 Times Evil,2016,R,NaN,NaN,English,90,None,None,/tcdAJMNPAlqtsNPtYK0F0GUgHKG.jpg,Documentary,History is replete with psychotic killers with...,NaN,NaN,NaN,NaN,"{""reelgood"": ""13-times-evil-2016"", ""amazon"": ""..."
2,341957,21 Days,21 Days,2014,R,"Nopal Army, Tortilla Flats Productions",USA,English,89,Kathleen Behun,"Max Hambleton, Whitney Rose Pynn, Mickey River",/1IXt1E6i6tH8gaCFG6jhlsKM4V5.jpg,"Thriller, Horror",Three filmmakers embark on a paranormal challe...,NaN,NaN,4.5,493.0,"{""reelgood"": ""21-days-2014"", ""amazon"": ""B01N9Z..."
3,441453,36 Hour Layover,36 Hour Layover,2016,None,RLJ Entertainment,USA,English,88,Mark Harris,None,/vZCoqQecIEGqfDr4Q3Ppvg3ZXIF.jpg,"Comedy, Romance",This film is a romantic comedy about a steward...,1000000.0,NaN,5.8,74.0,"{""reelgood"": ""36-hour-layover-2016"", ""amazon"":..."
4,270802,666: Devilish Charm,666: Devilish Charm,2014,None,Rapid Heart Pictures,USA,English,78,"David DeCoteau, Rolfe Kanefsky","R.J. Cantu, Ryan Walker",/cauliUwjBwVEFWGBhlpjnBmWec5.jpg,"Thriller, Horror",When a cursed devil's charm' bracelet shows up...,1000000.0,NaN,3.4,110.0,"{""reelgood"": ""666-devilish-charm-2014"", ""amazo..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 0 to 318
Data columns (total 19 columns):
Movie_ID         319 non-null int64
Title            319 non-null object
OriginalTitle    319 non-null object
Year             319 non-null object
Rating           149 non-null object
Companies        295 non-null object
Country          314 non-null object
Language         319 non-null object
RunTime          319 non-null int64
Crew             301 non-null object
Cast             256 non-null object
Poster           319 non-null object
Genres           318 non-null object
Synopsis         319 non-null object
Budget           84 non-null float64
Gross            102 non-null float64
ScoreImdb        312 non-null float64
VotesImdb        312 non-null float64
Indeces          319 non-null object
dtypes: float64(4), int64(2), object(13)
memory usage: 47.5+ KB


In [18]:
# for development, return the merged dataframe

df = MD.Editor.RunMaster Movies()
df[150:152]

SyntaxError: invalid syntax (<ipython-input-18-4a2bd81442f3>, line 3)

In [ ]:
df.info()

In [ ]:
df.loc[list(range(140,160)), ['Title_tmdb', 'Score_tmdb', 'Votes_tmdb', 'Score_imdb', 'Votes_imdb']]

In [ ]:
df['Rating_rlgd'].value_counts()

In [ ]:
tmdb_genre = []
rlgd_genre = []
imdb_genre = []
for idx, row in df.iterrows():
    tmdb = row['Genres_tmdb'] or ''
    rlgd = row['Genres_rlgd'] or ''
    imdb = row['Genres_imdb'] or ''
    
    tmdb_genre += tmdb.split(', ') if type(tmdb)==str else []
    rlgd_genre += rlgd.split(', ') if type(rlgd)==str else []
    imdb_genre += imdb.split(', ') if type(imdb)==str else []
    
    tmdb_ct = PD.Series(tmdb_genre).value_counts()
    rlgd_ct = PD.Series(rlgd_genre).value_counts()
    imdb_ct = PD.Series(imdb_genre).value_counts()
    
#tmdb_ct

In [ ]:
df.loc[[111, 136, 148, 269, 272, 19, 44, 47], ['Title_tmdb', 'Budget_tmdb', 'Budget_imdb']]

In [ ]:
round( ((6.8 * 293) + (5.8 * 6457))/(293+6457) , 2)

df.dropna(subset=['Votes_imdb'])['Votes_tmdb'].astype('int').sum()
df.dropna(subset=['Votes_imdb'])['Votes_imdb'].astype('int').sum()

In [ ]:
MD.Editor.GetMasterMovie(df.iloc[56])

In [ ]:
MD.Editor.GetMasterMovie(df.iloc[166])